In [26]:
# word count
from pyspark import SparkContext

sc = SparkContext(master = 'local[4]')

In [27]:
import requests
url = "https://mas-dse-open.s3.amazonaws.com/Moby-Dick.txt"
local_path = "Moby-Dick.txt"
r = requests.get(url, allow_redirects=True)
open(local_path, 'wb').write(r.content)

1257260

In [28]:
text_file = sc.textFile("Moby-Dick.txt")
type(text_file)
print (text_file.take(1))

['The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville']


In [29]:
%%time
words = text_file.flatMap(lambda line: line.split(' '))
non_empty = words.filter(lambda x: x!='')
key_value = non_empty.map(lambda word: (word, 1))
counts = key_value.reduceByKey(lambda a,b: a+b)
Sum = counts.map(lambda x: x[1]).reduce(lambda x, y: x+y)
print (Sum)

215133
CPU times: user 10 ms, sys: 20 ms, total: 30 ms
Wall time: 960 ms


In [30]:
# most frequent words

#method 1 - on the head node
C = counts.collect()
C.sort(key = lambda x: x[1])
print ('top 5 most common words: ', C[-5:])



top 5 most common words:  [('to', 4510), ('a', 4533), ('and', 5951), ('of', 6587), ('the', 13766)]


In [35]:
#method 2 - collect at the end
reversed_counts = counts.map(lambda x: (x[1], x[0]))
sorted_counts = reversed_counts.sortByKey(ascending = False)
D = sorted_counts.take(5)
print ('most common words\n' + '\n'.join(['%d:\t%s'%c for c in D]))

most common words
13766:	the
6587:	of
5951:	and
4533:	a
4510:	to
